# **COMP561 - Fall 2024  - Homework#1**


# Question #1 - Global pairwise sequence alignment, with splippage-aware scoring

In this question, we implement a version of the Needleman-Wunsch algorithm, modified to take into account the splippage-aware scoring: Score(A) = same as standard linear gap penalty scoring scheme, except that:

1. if A[1,i] = ‘-‘, the score assigned to column i is:
cs if A[2,i]=A[2,i-1]
cn otherwise

2. if A[2,i] = ‘-‘, the score assigned to column i is:
cs if A[1,i]=A[1,i-1]
cn otherwise

# I - Needleman-Wunsch algorithm, with splipage aware penalty



To take into account this specific splippage-aware gap penalty, we need to bring the following changes to the Needleman-Wunsch algorithm:

1 - initialization:
X[0, 0] = 0
X[0, 1] = cn
For i>1, X[0, i] = X[0, i-1] + cs if A[2,i] = A[2,i-1], X[0, i-1] + cn otherwise
Similarly, X[1, 0] = cn
For j>1, X[j, 0] = X[j-1, 0] + cs if A[1,j] = A[1,j-1], X[j-1, 0] + cn otherwise

2 - Filling the score matrix:
similarly to N-W algorithm, there are always 3 cases to consider:

case1: alignment of the 2 elements (i, j): Score(i,j) = Score(i-1, j-1) + match or mismatch cost (same as N-W algo)
case2: gap in sequence1: Score(i,j) = Score(i-1, j) + gap penalty (with splippage)
case2: gap in sequence2: Score(i,j) = Score(i, j-1) + gap penalty(with splippage)

The only difference is on the calculation of the gap penalty with splippage, where we need to consider the previous 2 consecutive elements in the other sequence to determine which gap penalty (with splippage or without splippage) to apply.


In [1]:
import numpy as np

# Inputs
MATCH_SCORE = 1
MISMATCH_SCORE = -1
CS = -1  # Slippage gap penalty
CN = -2  # Non-slippage gap penalty


## step1: Populate Score and Traceback Matrices

In [2]:


# function that computes the score matrix and trace_back matrix from input data and 2 sequences


def needleman_wunsch_slippage(seq1, seq2, match_score=MATCH_SCORE, mismatch_score=MISMATCH_SCORE, cs=CS, cn=CN):

    # Initialize a score matrix of size (seq1 + 1 )* (seq2 + 1) with zeros
    n1 = len(seq1)
    n2 = len(seq2)
    score_matrix = np.zeros((n1 + 1, n2 + 1))
    # Initialize a traceback_matrix with zeros
    traceback_matrix = np.zeros((n1 + 1, n2 + 1))

    # Initialization: Fill the score matrix first row and first column with initial gap penalties (with splippage)
    score_matrix[1, 0] = cn
    score_matrix[0, 1] = cn
    traceback_matrix[1, 0] = 2  # (gap in seq2)
    traceback_matrix[0, 1] = 3  # (gap in seq1)

    for i in range(2, n1 + 1):
        traceback_matrix[i, 0] = 2  # (gap in seq2)
        if seq1[i - 1] == seq1[i - 2]:
            score_matrix[i, 0] = score_matrix[i - 1, 0] + cs  # Slippage gap penalty
        else:
            score_matrix[i, 0] = score_matrix[i - 1, 0] + cn  # Non-slippage gap penalty
    for j in range(2, n2 + 1):
        traceback_matrix[0, j] = 3 #(gap in seq1)
        if seq2[j - 1] == seq2[j - 2]:
            score_matrix[0, j] = score_matrix[0, j - 1] + cs  # Slippage gap penalty
        else:
            score_matrix[0, j] = score_matrix[0, j - 1] + cn  # Non-slippage gap penalty



    # Populate the score matrix (same algo as Needleman-Wunsch, except for the calculation of the penalty)
    for i in range(1, n1 + 1):
        for j in range(1, n2 + 1):
            # 3 cases:
            #Case # 1 : align position i in seq 1 and position j:
            align = score_matrix[i - 1, j - 1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            #Case2: align seq1 position i with a gap
            gap1 = score_matrix[i - 1, j] + cn  # Default non-slippage gap penalty
            #Case3: align seq2 position j with a gap
            gap2 = score_matrix[i, j - 1] + cn  # Default non-slippage gap penalty

            if i > 1 and seq1[i - 2] == seq1[i - 1]:
                gap1 = score_matrix[i - 1, j] + cs  # Slippage gap penalty
            if j > 1 and seq2[j - 2] == seq2[j - 1]:
                gap2 = score_matrix[i, j - 1] + cs  # Slippage gap penalty

            score_matrix[i, j] = max(align, gap1, gap2)

              # Track the optimal choice for traceback
            if score_matrix[i, j] == align:
                traceback_matrix[i, j] = 1  # Diagonal (match/mismatch)
            elif score_matrix[i, j] == gap1:
                traceback_matrix[i, j] = 2  # Up (gap in seq2)
            else:
                traceback_matrix[i, j] = 3  # Left (gap in seq1)
    return score_matrix, traceback_matrix





## step2 : Retrieve optimal alignment from traceback matrix

In [3]:
# Function to find optimal alignments from the traceback matrix
def traceback(score_matrix, traceback_matrix, seq1, seq2):
    n1 = len(seq1)
    n2 = len(seq2)
    # Traceback process to create the aligned sequences
    # start with empty strings for align1 and align2
    align1, align2 = '', ''
    #start to read the traceback matrix at the end (rightest and bottom position), i.e. position (i,j) = (n1, n2)
    i, j = n1, n2

    while i > 0 or j > 0:
        if traceback_matrix[i, j] == 1:  # Diagonal (match/mismatch)
            align1 += seq1[i - 1]
            align2 += seq2[j - 1]
            i -= 1
            j -= 1

        elif traceback_matrix[i, j] == 2:  # Up (gap in seq2)
            align1 += seq1[i - 1]
            align2 += '-'
            i -= 1

        else:  # Left (gap in seq1)
            align1 += '-'
            align2 += seq2[j - 1]
            j -= 1

    # Reverse the aligned sequences as they were built backwards
    align1 = align1[::-1]
    align2 = align2[::-1]

    return align1, align2, score_matrix[n1, n2]

In [4]:

def traceback(score_matrix, traceback_matrix, seq1, seq2):
    n1 = len(seq1)
    n2 = len(seq2)

    # Initialize an empty alignment matrix with 2 rows (number of columns will be the length of alignment, not known initially)
    A = [[], []]  # Row 1 :seq1 alignment, Row 2: seq2 alignment

    # Start to read the traceback matrix at the end (position (i,j) = (n1, n2))
    i, j = n1, n2

    while i > 0 or j > 0:
        if traceback_matrix[i, j] == 1:  # Diagonal (match/mismatch)
            A[0].append(seq1[i - 1])
            A[1].append(seq2[j - 1])
            i -= 1
            j -= 1

        elif traceback_matrix[i, j] == 2:  # Up (gap in seq2)
            A[0].append(seq1[i - 1])
            A[1].append('-')
            i -= 1

        else:  # Left (gap in seq1)
            A[0].append('-')
            A[1].append(seq2[j - 1])
            j -= 1

    # Reverse each row since the traceback process builds the alignment backwards
    A[0][::-1]
    A[1][::-1]

    return A, score_matrix[n1, n2]


# II - Helper functions to read input data


## step1: retrieve the fasta files from the url addresses

We'll first retrieve the 2 fasta files from the url listed in the homework.

In [5]:
#Import the urlretrieve function
from urllib.request import urlretrieve

#url addresses of the 2 fasta files
url1 = 'http://www.cs.mcgill.ca/~blanchem/561/hw1_brca1_3utr_small.fa'
url2 = 'http://www.cs.mcgill.ca/~blanchem/561/hw1_brca1_3utr_full.fa'

#Name of the files to save the data
filename1 = 'hw1_brca1_3utr_small.fasta'
filename2 = 'hw1_brca1_3utr_full.fasta'



#Download the data
urlretrieve(url1, filename1)
urlretrieve(url2, filename2)



('hw1_brca1_3utr_full.fasta', <http.client.HTTPMessage at 0x7a9e1e32dcf0>)

## step 2: parse the fasta files

Then, we'll parse the fasta files to extract the information in a dictionary

In [6]:
# Function that extracts the data from an input fasta file into a dictionary
def parse_fasta_file(input_file):
    """Return a dict of {id:gene_seq} pairs based on the sequences in the input FASTA file
    input_file -- input fasta file
    """
    parsed_seqs = {}
    curr_seq_id = None
    curr_seq = []
    f = open(input_file)
# read the file line by line and
    for line in f:
        line = line.strip()
        # for each new indicator of new sequence id:
        if line.startswith(">"):
            if curr_seq_id is not None:
                parsed_seqs[curr_seq_id] = ''.join(curr_seq)

            curr_seq_id = line[1:]
            curr_seq = []
            continue

        curr_seq.append(line)

    #Add the final sequence to the dict
    parsed_seqs[curr_seq_id] = ''.join(curr_seq)
    return parsed_seqs

This parsing function will be used to test the alignment algorithm on  the 2 DNA sequences provided in each Fasta files (see next Section III).

# III - Test the algorithm

Putting every pieces together: let's define a function that takes the five following arguments: \\
(1) File containing the two sequences to be aligned (FASTA format). \\

(2) The score for matches; \\

(3) The score for mismatches (assuming that all mismatches are
scored identically); \\

(4) The slippage gap penalty cs; \\

(5) The non-slippage gap penalty cn. \\
and prints out the optimal alignment score, and the optimal alignment
itself.

In [7]:
import os
def align_sequences(input_file, match_score, mismatch_score, cs, cn):
  # parse the fasta file
  f = open(input_file)
  parsed_seqs = parse_fasta_file(input_file)
  key1 = list(parsed_seqs.keys())[0]
  key2 = list(parsed_seqs.keys())[1]
  # save the sequences in string variables seq1, seq2
  seq1 = parsed_seqs[key1]
  seq2 = parsed_seqs[key2]

  #compute the Needleman-Wunsch score and traceback matrices
  score_matrix, traceback_matrix = needleman_wunsch_slippage(seq1, seq2, match_score, mismatch_score, cs, cn)

  #find the optimal alignment, based on the traceback matrix
  alignment, score = traceback(score_matrix, traceback_matrix, seq1, seq2)
  # Convert matrix rows to strings for output
  align1 = ''.join(alignment[0])  # First row for Sequence 1
  align2 = ''.join(alignment[1])  # Second row for Sequence 2

  # Print the alignment and its score
  print(f"Alignment score: {score}")
  print(f"\nOptimal Alignment :")
  print(f"{key1}: {align1}")  # Print key1 and align1 on a new line
  print(f"{key2}: {align2}")  # Print key2 and align2 on a new line

  # Save the alignment and its score in a text file
  base_file_name = os.path.splitext(os.path.basename(input_file))[0] + ".txt"

    # Define output file name
  output_file = "alignment_output_" + base_file_name
  with open(output_file, 'w') as f_out:
        f_out.write(f"Alignment score: {score}\n")
        f_out.write(f"\nOptimal Alignment :\n")
        f_out.write(f"{key1}: {align1}\n")
        f_out.write(f"{key2}: {align2}\n")

  print(f"\nAlignment saved in: {output_file}")

## Test1: first DNA sequences (FASTA file : hw1_brca1_3utr_small.fa)

In [8]:
align_sequences(filename1, match_score= 1, mismatch_score=-1, cs=-1, cn=-2)

Alignment score: 119.0

Optimal Alignment :
human_BRCA1_3UTR: CTTCACAAACGATGGTTCAAATAAACGTCACAATTGTCGTGTTGTAAATGTTTTGCATAAAACATGTTAGTTCAGAAGTGACGGGAACGTGTGACCCCCC-CGATCCCTTCTGGATCAGGAAGGTTGTCGATATTTGTCAGGACCTATTACCCAAATACTTTTTGTGAAAAAGAAGGAAGTCGTTCGTTTTAATAAATAC-T---
mouse_BRCA1_3UTR: ----ACAAACGATAAATCAAATAAACGTCATAATTGTCGTGTCGTAAATATT-TGTATGAAATATATTAGTCCAGAGGTAACGGGAACGTATGACCCCCCCCGGTCTCTTCTGGATGAGGGGGGTTATAGATATTTGTCAGGTTCTAATACTCAAGTACTTTTTATACAAGAGAAGGAAGTCGTTCGTTTTAATAAATACTTCAT

Alignment saved in: alignment_output_hw1_brca1_3utr_small.txt


## Test2: second DNA sequences (FASTA file : hw1_brca1_3utr_full.fa)

In [9]:
align_sequences(filename2, match_score= 1, mismatch_score=-1, cs=-1, cn=-2)

Alignment score: 215.0

Optimal Alignment :
human_BRCA1_3UTR: CTTCACAAACGATGGTTCAAATAAACGTCACAATTGTCGTGTTGTAAATGTTTTGCATAAAACATGTTAGTTCAGAAGTGACGGGAACGTGTGACCCCCC-CGATCCCTTCTGGATCAGGAAGGTTGTCGATATTTGTCAGGACCTATTACCCAAATACTTTTTGTGAAAAAGAAGGAAGTCGTTCGTTTTAATAAATACTTCGACATACCAAAGTCGTTGTCCCTCGTT-TCCTTTTTTTAGTGGAGTTTCTTTCGTTGTCGAAGGA-AGGACCACCCTAGACAGTAAAATATCTATACTTTATAAGTACGGTCTCCA-GAATATAAAATTCTCCTTACCTAATATATGGTCT-CGATGTTGT-TATTTGTAAAATGAATAAT-GATTACTCCTTAATCTTCTGACAGAAACCTTTGGCCAAGAACTTTTAGAAGACGACAAAATCTTGTGTAAGAAATCTTTAGATCGTTTATATAGAGTCTGAAAATCTTTAGAGAAGATCAAAGTAAAAGGAAAAAAAAAAAAAAAAAAACTCGGTGTCAGAGTGACAGTGGGTCCGACCTCACGGCACCATACTAGAACCGAGTGACGTTGGAGG-TGGAGGGCCCGACTTCACTAAGAGGACGGAATCGGTGGACTCATCGACCCTAATGTCCACAGGTGGTGGTACTGGCCGATTAAAGACATAAAAATCATCTCTACCCCAAAGTGGTACAACCGGTCCGACCA-AAGCTTGAGGAC-TG--GAGGTCACTAGACGGGTGGA-ACCGGAGGGTTTCACGACCCTAATGTCCGCACTCGGTGGTACGGGTCCAAAGTTCAAA-GGAAAAGTAAAGATTATGGACGGA-G-TCTTAAAGGAGG-GGT-TACAAGGT-G-AGGTTG-TAA--A--CTCT-T-GA-CGGGTTC-C---TGATAAGACTGAAATT